## interfaceの同定

In [ ]:
import gemmi

# 構造ファイルを読み込み（cif, bcif, pdbに対応）
structure = gemmi.read_structure("1bh8-assembly1.cif.gz")
model = structure[0]

# 対象チェーンを指定
chain_A = model["A"]
chain_B = model["B"]

# 標準アミノ酸の3文字表記セット
standard_amino_acids = {
    "ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY",
    "HIS", "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER",
    "THR", "TRP", "TYR", "VAL"
}

# 結果格納用のセット（重複防止）
interface_residues_A = set()
interface_residues_B = set()

# 距離しきい値（Å）
threshold = 5.0

# 距離に基づくインターフェース判定
for resA in chain_A:
    if resA.name not in standard_amino_acids:
        continue
    for atomA in resA:
        posA = atomA.pos
        for resB in chain_B:
            if resB.name not in standard_amino_acids:
                continue
            for atomB in resB:
                posB = atomB.pos
                if posA.dist(posB) <= threshold:
                    interface_residues_A.add((chain_A.name, resA.seqid.num, resA.name))
                    interface_residues_B.add((chain_B.name, resB.seqid.num, resB.name))
                    break

# 🔽 出力（チェーンID → 残基番号の順にソート）
def print_interface_residues(title, residues):
    print(title)
    for chain_id, res_num, res_name in sorted(residues, key=lambda x: (x[0], x[1])):
        print(f"{chain_id} {res_name} {res_num}")
    print()

print_interface_residues("Interface residues in chain A:", interface_residues_A)
print_interface_residues("Interface residues in chain B:", interface_residues_B)


## Interfaceの面積を計算

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import tempfile
import freesasa
import gemmi

STANDARD_AA = {
    "ALA","ARG","ASN","ASP","CYS","GLN","GLU","GLY",
    "HIS","ILE","LEU","LYS","MET","PHE","PRO","SER",
    "THR","TRP","TYR","VAL"
}

def build_new_structure_from_chains(src: gemmi.Structure, chain_ids):
    """
    指定されたチェーンIDと標準アミノ酸のみを含む新しいStructureオブジェクトを生成します。
    gemmi.Structure.clone() を使用して安全にコピーし、不要なチェーンと残基を削除します。
    """
    new_s = src.clone()

    # If the structure is empty, return it as is.
    if not new_s:
        return new_s

    model = new_s[0]

    # Collect chain names to remove.
    chains_to_remove = [c.name for c in model if c.name not in chain_ids]

    # Remove unwanted chains.
    for chain_name in chains_to_remove:
        model.remove_chain(chain_name)

    # Remove non-standard amino acids from the remaining chains.
    for chain in model:
        # We need to collect indices and remove them in reverse order
        # to avoid issues with changing list size during iteration.
        residues_to_remove_indices = [
            i for i, res in enumerate(chain) if res.name not in STANDARD_AA
        ]
        for i in sorted(residues_to_remove_indices, reverse=True):
            del chain[i]

    return new_s

def structure_total_sasa_with_freesasa(structure: gemmi.Structure) -> float:
    """
    gemmi.Structure -> 一時PDB -> freesasa.calc() -> 総ASA（Å²）
    """
    tmp_name = None
    try:
        with tempfile.NamedTemporaryFile(suffix=".pdb", delete=False) as tmp:
            tmp_name = tmp.name
        structure.write_pdb(tmp_name)
        st = freesasa.Structure(tmp_name)
        result = freesasa.calc(st)
        return result.totalArea()
    finally:
        if tmp_name and os.path.exists(tmp_name):
            os.remove(tmp_name)

def compute_interface_bsa(input_structure_path: str, chainA: str, chainB: str):
    """
    BSA = ((ASA_A + ASA_B) - ASA_AB) / 2 を計算
    gemmi の “コピー” 系 API を使わず、構造を作り直して freesasa に渡す
    """
    src = gemmi.read_structure(input_structure_path)

    # AB複合体
    struct_AB = build_new_structure_from_chains(src, [chainA, chainB])
    asa_AB = structure_total_sasa_with_freesasa(struct_AB)

    # A単独
    struct_A = build_new_structure_from_chains(src, [chainA])
    asa_A = structure_total_sasa_with_freesasa(struct_A)

    # B単独
    struct_B = build_new_structure_from_chains(src, [chainB])
    asa_B = structure_total_sasa_with_freesasa(struct_B)

    bsa = 0.5 * ((asa_A + asa_B) - asa_AB)
    return bsa, (asa_A, asa_B, asa_AB)

if __name__ == "__main__":
    cif_path = "1bh8-assembly1.cif.gz"   # 例
    chainA, chainB = "A", "B"

    bsa, (asa_A, asa_B, asa_AB) = compute_interface_bsa(cif_path, chainA, chainB)
    print(f"ASA(A):  {asa_A:.2f} Å²")
    print(f"ASA(B):  {asa_B:.2f} Å²")
    print(f"ASA(AB): {asa_AB:.2f} Å²")
    print(f"Interface (BSA) between {chainA}-{chainB}: {bsa:.2f} Å²")
